# LLM from scratch

Project based on: https://www.youtube.com/watch?v=UU1WVnMk4E8 

Dorothy and the Wizard in OZ soruce: https://gutenberg.org/cache/epub/22566/pg22566.txt

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


hyperparameters

In [42]:
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

In [5]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [6]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
print(encoded_hello)
decoded_hello = decode(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


Using torch

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [10]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:


tensor([[ 1, 54, 67, 57,  1, 65, 54, 71],
        [62, 77,  1, 56, 68, 65, 68, 71],
        [11,  3,  0,  0,  3, 33,  1, 72],
        [ 1, 69, 68, 56, 64, 58, 73, 23]], device='cuda:0')
targets:
tensor([[54, 67, 57,  1, 65, 54, 71, 60],
        [77,  1, 56, 68, 65, 68, 71, 58],
        [ 3,  0,  0,  3, 33,  1, 72, 61],
        [69, 68, 56, 64, 58, 73, 23,  1]], device='cuda:0')


In [40]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            # batch size, time, channels (number of classes, vocabulary size)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


igZY O
pn!ZEvUfFQJ? qwA?DwzWdwW2QahxWA-d&Dw﻿]a;
apg(hWah]pYCYn;QaIp18qiSy-;Z_9Y:Wr,.anEVUN.5 *7GvH6JA;*Vh)O9H2Ow?B5tRYJFg﻿*C806M58yKJ'R
?xv.[rvndC﻿i
2Oi;lOb_yAahWlX8Zu﻿bmxu69
I3xqw1ahH)wAqrvFF?A,eg&﻿l(oQ3E-km[.ctewAy&jSJ3LQfC?_vM[qj
rNKJQiajf3l23( JaMzt_IUnx4rH'!g(0sd)88OSg﻿LAkr9rpMxQ:a_yWI"c﻿A﻿fct[q[JuC3m[F3ftksKm&IK9DD QD4x4UOA?7RvhWlJG(kf!Ruzp(j'Q5?mya?8swxM))-.ZW&&z?&q6dJFC.K_D,;OWdwY﻿y3Q5( dR[*qoK;U﻿pr2myt,RYqE11Z_9Yo3D,Qods5b﻿f:nd&rH''(a﻿pBv29.KVC)OHLJBnwgZ3r_-G6q_qzVzN(jND((TwUn-2T6&B!dIz


In [49]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.505, val loss: 2.530
step: 250, train loss: 2.498, val loss: 2.553
step: 500, train loss: 2.458, val loss: 2.547
step: 750, train loss: 2.472, val loss: 2.511
step: 1000, train loss: 2.466, val loss: 2.531
step: 1250, train loss: 2.483, val loss: 2.526
step: 1500, train loss: 2.468, val loss: 2.511
step: 1750, train loss: 2.486, val loss: 2.546
step: 2000, train loss: 2.471, val loss: 2.518
step: 2250, train loss: 2.488, val loss: 2.529
step: 2500, train loss: 2.482, val loss: 2.528
step: 2750, train loss: 2.486, val loss: 2.504
step: 3000, train loss: 2.486, val loss: 2.496
step: 3250, train loss: 2.460, val loss: 2.513
step: 3500, train loss: 2.468, val loss: 2.505
step: 3750, train loss: 2.478, val loss: 2.520
step: 4000, train loss: 2.462, val loss: 2.526
step: 4250, train loss: 2.463, val loss: 2.497
step: 4500, train loss: 2.453, val loss: 2.503
step: 4750, train loss: 2.449, val loss: 2.531
step: 5000, train loss: 2.449, val loss: 2.542
step: 5250, train l

In [50]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


t hape, ay."Nom e ord M'p
g m I hirugoose t ass l ordialy." toucasly ie heh cinors "
WDizbrt the, thast burirr areda hee DE.

 lanu
ssq4)X()Jimanostoftirouloind tele thathe tch piked poopsed we

"Wghedf " tigngugy the n me d arastime, hono t indesked wiooplarouedesthod wrsaga t g


t ind ad asthin's gsparowngrstthinds beaniv," and-z!urcus sid t p m. t?e f fe Sed meple

CH2m t, whe'mpainchellit th as whast swave " fuman
PRme t ca IUSingen bs.
seratollite oy, ft
s Z
"hiz in
"B?"had w
"BEXpress mos
